In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
kartik2112_fraud_detection_path = kagglehub.dataset_download('kartik2112/fraud-detection')

print('Data source import complete.')


In [ ]:
# Standard libraries
import pandas as pd
from scipy.stats import chi2_contingency

# Enable auto-reload for module reloading during development
%load_ext autoreload
%autoreload 2

# Set display options for Pandas to show all columns
pd.set_option('display.max_columns', None)

# Load custom scripts
import scripts.eda_functions as eda
import scripts.data_cleaning as dc
import scripts.data_check as ch


# Data Quality Analysis (Data Cleaning and Manipulation)

In [ ]:
# Load dataset
transactions = pd.read_csv('data/fraudTest.csv')

In [ ]:
# First look at the first few rows of the dataset
transactions.head(5)

In [ ]:
# Check data types, number of missing values, duplicates, and unique values
ch.check(transactions)

# Good news: There are no missing values or duplicate records!

In [ ]:
# Formatting column names using snake case
transactions_clean = dc.snake(transactions)

# Dropping unnecessary columns:
# - 'unnamed:_0' is redundant as it is equivalent to the index column.
# - 'unix_time' is not needed since the transaction date and time are already in a separate column.
transactions_clean = dc.drop_col(transactions_clean, ["unnamed:_0", "unix_time", "trans_num"])

# Removing the word 'fraud_' from the beginning of the merchant names
transactions_clean = dc.remove_prefix_from_column(transactions_clean, 'merchant', 'fraud_')

# Cleaning the 'category' column by removing underscores and capitalizing the names
transactions_clean = dc.clean_category_column(transactions_clean, 'category')

# Gender in long format
transactions_clean = dc.clean_gender_column(transactions_clean)

# State in long format (expanding abbreviations)
transactions_clean = dc.replace_state_abbreviations(transactions_clean)

# Convert 'trans_date_trans_time' and 'dob' (date of birth) to datetime format
transactions_clean = dc.convert_to_datetime(transactions_clean, ['trans_date_trans_time', 'dob'])

# Add age column based on 'dob'
transactions_clean = dc.age_calculator(transactions_clean)

# Adding hour, day, month, day of the week columns to the data frame
transactions_clean = dc.preprocess_datetime(transactions_clean)

# Let's check these changes
transactions_clean.head(5)

In [ ]:
# Re-checking the data types of the transactions_clean DataFrame
ch.check(transactions_clean)

In [ ]:
# Save the cleaned DataFrame to a new CSV file
transactions_clean.to_csv('data/cleaned_transactions.csv', index=False)

In [ ]:
# Save a copy for Tableau
transactions_clean.to_csv('data/fraud_detection_project_tableau.csv', index=True)

In [ ]:
# Import the cleaned CSV file into a new DataFrame
df = pd.read_csv('data/cleaned_transactions.csv')


In [ ]:
# A look at the first few rows of the cleaned dataset
df.head(5)

In [ ]:
# Convert 'trans_date_trans_time' and 'dob' (date of birth) to datetime in the new DataFrame
df = dc.convert_to_datetime(df, ['trans_date_trans_time', 'dob'])

# Convert date_of_week to a categorica column in the new DataFrame
df = dc.preprocess_datetime(df)

In [ ]:
# Re-checking the data types of the cleaned DataFrame
ch.check(df)

# Analysis of the Data (EDA and Descriptive Statistics)

## Fraud vs. Non-Fraud Analysis

### 1.  What percentage of transactions in the dataset are fraudulent?

In [ ]:
frequency_table, frequency_table_total = eda.calculate_fraud_percentage(df)
print(f'The percentage of fraud is {frequency_table["Relative frequency"].iloc[0]} % '
      f'and the percentage of non-fraud is {frequency_table["Relative frequency"].iloc[1].round(2)} %')
frequency_table_total

In [ ]:
eda.plot_fraud_percentage(df)

In [ ]:
eda.plot_fraud_frequency_by_card(df)

### 2.  What are the average amounts of fraud vs. non-fraud transactions?

In [ ]:
avg_transaction_amounts = eda.analyze_transaction_amounts(df)
avg_transaction_amounts

In [ ]:
fraud_stats = eda.fraud_statistics(df)
fraud_stats

In [ ]:
eda.plot_fraud_transaction_amounts(df, False)

In [ ]:
eda.plot_fraud_transaction_amounts(df, True)

In [ ]:
# Calculate the Skewness for Non-Fraud Transactions
df[df['is_fraud'] == 'Non-Fraud']['amt'].skew().round(2)

# The distribution is positively skewed (or right-skewed), meaning there is a longer tail on the right side of the distribution.
# The mass of the distribution is concentrated on the left of the figure with large amount of outliers with large values


In [ ]:
# Calculate the Kurtosis for Non-Fraud Transactions
df[df['is_fraud'] == 'Non-Fraud']['amt'].kurtosis().round(2)

# Kurtosis > 0 : The distribution is leptokurtic.
# It has fatter tails and a sharper peak than the normal distribution, indicating more extreme values (outliers) than expected in a normal distribution.

## Fraud Trends Over Time

### 3.  Do fraud transactions spike at certain hours of the day?

In [ ]:
# Non-Fraud and Fraud Transactions counts by hour
fraud_counts = eda.fraud_trends(df, 'hour')
fraud_counts

In [ ]:
# Mean of Non-Fraud and Fraud Transactions by hour
fraud_counts = eda.fraud_trends(df, 'hour','mean')
fraud_counts

In [ ]:
eda.plot_fraud_trends(df, 'hour')

In [ ]:
eda.plot_fraud_rate_by_time(df, 'hour')


In [ ]:
eda.plot_fraud_trends(df, 'hour', 'mean')

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'hour', 'mean')

In [ ]:
eda.plot_fraud_boxplots(df)

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'hour', 'mode')

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'hour', 'max')

In [ ]:
eda.plot_fraud_trends(df, 'hour', 'max')

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'hour', 'min')

### 4. Do fraud transactions spike at certain day of the month?

In [ ]:
fraud_counts = eda.fraud_trends(df, 'day')
fraud_counts

In [ ]:
eda.plot_fraud_trends(df, 'day')

In [ ]:
eda.plot_fraud_rate_by_time(df, 'day')

In [ ]:
eda.plot_fraud_trends(df, 'day', 'mean')

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'day', 'mean')

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'day', 'median')

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'day', 'mode')

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'day', 'max')

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'day', 'min')

In [ ]:
eda.plot_fraud_heatmap(df, 'day', 'hour')

In [ ]:
eda.plot_fraud_heatmap_statistic(df, 'day', 'hour','mean')

### 5. Do fraud transactions spike at certain day of the week?

In [ ]:
fraud_counts = eda.fraud_trends(df, 'day_of_week')
fraud_counts

In [ ]:
eda.plot_fraud_trends(df, 'day_of_week')

In [ ]:
eda.plot_fraud_rate_by_time(df, 'day_of_week')

In [ ]:
eda.plot_fraud_trends(df, 'day_of_week', 'mean')

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'day_of_week', 'mean')

In [ ]:
eda.plot_fraud_boxplots_time(df, 'day_of_week')

In [ ]:
eda.plot_fraud_trends(df, 'day_of_week', 'median')

In [ ]:
eda.plot_fraud_trends(df, 'day_of_week', 'mode')

In [ ]:
eda.plot_fraud_trends(df, 'day_of_week', 'max')

In [ ]:
eda.plot_fraud_trends(df, 'day_of_week', 'min')

In [ ]:
eda.plot_fraud_heatmap(df, 'day_of_week', 'hour')

In [ ]:
# Chi-square tests to test the independence of time and hour in the fraud rates
crosstab_result =  pd.crosstab(df[df['is_fraud'] == 1]['hour'], df[df['is_fraud'] == 1]['day_of_week'])
chi2_stats, chi2_pvalue, _, _ = chi2_contingency(crosstab_result)
chi2_pvalue

# A p-value > 0.05, indicates that there is not enough evidence to conclude a significant association between the variables.

In [ ]:
eda.plot_fraud_heatmap_statistic(df, 'day_of_week', 'hour','mean')

In [ ]:
# Correlation matrix to visualizes correlation coefficients between pairs of continuous variables using color gradients.
eda.plot_correlation_heatmap(df)

### 6.  Are there seasonal trends in fraud?

In [ ]:
fraud_counts = eda.fraud_trends(df, 'month')
fraud_counts

In [ ]:
eda.plot_fraud_trends(df, 'month')

In [ ]:
eda.plot_fraud_rate_by_time(df, 'month')

In [ ]:
eda.plot_fraud_trends(df, 'month', 'mean')

In [ ]:
eda.plot_fraud_trends_same_axes(df, 'month', 'mean')

In [ ]:
eda.plot_fraud_boxplots_time(df, 'month')

In [ ]:
eda.plot_fraud_trends(df, 'month', 'median')

In [ ]:
eda.plot_fraud_trends(df, 'month', 'mode')

In [ ]:
eda.plot_fraud_trends(df, 'month', 'max')

In [ ]:
eda.plot_fraud_trends(df, 'month', 'min')

In [ ]:
eda.plot_fraud_heatmap(df, 'month', 'hour')

In [ ]:
eda.plot_fraud_heatmap(df, 'month', 'day_of_week')

In [ ]:
eda.plot_fraud_heatmap_statistic(df, 'month', 'day_of_week', 'mean')

##  Risky Transaction Types

### 7. What merchant category have the highest fraud rates?

In [ ]:
stats_df = eda.create_fraud_statistics_table(df)
stats_df

In [ ]:
eda.plot_high_risk_transaction_types(df, 'merchant')

In [ ]:
eda.plot_high_risk_transaction_types(df, 'category')

In [ ]:
eda.plot_high_risk_transaction_types_fre(df, 'category')

In [ ]:
fraud_counts = eda.plot_donut_histogram_by_net_pos_fraud_only(df, 'category')
fraud_counts

### 8. How do fraud and non-fraud transactions compare across merchant categories?

In [ ]:
eda.plot_fraud_vs_nonfraud_means(df, 'category')

In [ ]:
eda.plot_fraud_boxplots_time(df, 'category')

### 9. When do fraud transactions occur most often in each category?

In [ ]:
eda.plot_fraud_heatmap(df, 'category', 'hour')

In [ ]:
eda.plot_fraud_heatmap(df, 'category', 'day')

In [ ]:
eda.plot_fraud_heatmap(df, 'category', 'day_of_week')

In [ ]:
eda.plot_fraud_heatmap(df, 'category', 'month')

In [ ]:
eda.plot_fraud_heatmap_statistic(df, 'category', 'month', 'mean')

## Geographical Fraud Distribution

### 10. Do fraud transactions happen more in specific locations?

In [ ]:
eda.plot_high_risk_transaction_types(df, 'city')

In [ ]:
eda.plot_high_risk_transaction_types_fre(df, 'city')

In [ ]:
eda.plot_high_risk_transaction_types(df, 'state')

In [ ]:
eda.plot_high_risk_transaction_types_fre(df, 'state')

In [ ]:
eda.plot_fraud_vs_nonfraud_means(df, 'city')

In [ ]:
eda.plot_fraud_vs_nonfraud_means(df, 'state')

In [ ]:
eda.plot_fraud_heatmap(df, 'state', 'hour')

## Customer Characteristics and Fraud

### 11. Are certain users or accounts more frequently targeted?

In [ ]:
eda.plot_gender_fraud_and_box_plots(df)

In [ ]:
eda.plot_fraud_vs_nonfraud_age(df)

In [ ]:
eda.plot_fraud_vs_nonfraud_age_violin(df)

In [ ]:
eda.plot_age_grouped_fraud_nonfraud(df)
